<a href="https://colab.research.google.com/github/pedro-arruda09/mlp-backpropagation/blob/main/MLP/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!git clone https://github.com/valmirf/redes_neurais_pos.git

fatal: destination path 'redes_neurais_pos' already exists and is not an empty directory.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation.

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [23]:
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [24]:
import numpy as np
import random

# --- Funções de ativação (com sigmoid numericamente estável) ---
def sigmoid(z):
    # implementação numericamente estável
    return np.where(z >= 0,
                    1.0 / (1.0 + np.exp(-z)),
                    np.exp(z) / (1.0 + np.exp(z)))

def sigmoid_prime(z):
    s = sigmoid(z)
    return s * (1 - s)

def relu(z):
    return np.maximum(0, z)

def relu_prime(z):
    return (z > 0).astype(float)

# --- Classe Network corrigida ---
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)             # número de camadas (inclui entrada)
        self.sizes = sizes
        # biases: não existe bias para camada de entrada, por isso sizes[1:]
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # He initialization para ReLU (sqrt(2/x))
        self.weights = [np.random.randn(y, x) * np.sqrt(2.0/x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, x):
        """Retorna a saída da rede usando ReLU nas camadas ocultas e sigmoid na saída."""
        activation = x
        for i, (b, w) in enumerate(zip(self.biases, self.weights)):
            net = np.dot(w, activation) + b
            # se não for a última camada -> ReLU, senão -> sigmoid
            if i < len(self.weights) - 1:
                activation = relu(net)
            else:
                activation = relu(net)
        return activation

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None):
        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂)

            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j, acc, n_test, (acc*100)/n_test))
            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂):
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        self.weights = [w - (𝜂/len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases  = [b - (𝜂/len(mini_batch)) * nb for b, nb in zip(self.biases,  nabla_b)]

    def backprop(self, x, y):
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # --- Feedforward consistente (ReLU nas ocultas, sigmoid na saída) ---
        activation = x
        activations = [x]   # lista de ativações (pós-função) camada-a-camada
        nets = []           # lista de nets (w·a + b) camada-a-camada

        for i, (b, w) in enumerate(zip(self.biases, self.weights)):
            net = np.dot(w, activation) + b
            nets.append(net)
            # aplicar função correta dependendo se é última camada
            if i < len(self.weights) - 1:
                activation = sigmoid(net)
            else:
                activation = sigmoid(net)
            activations.append(activation)

        # --- Backward pass ---
        # erro da última camada: (output - y) * f'(net_output)
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        # backprop pelas camadas ocultas
        # l = 2 é penúltima camada, etc.
        for l in range(2, self.num_layers):
            net = nets[-l]
            sp = sigmoid_prime(net)  # derivada de ReLU para camadas ocultas
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())

        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [25]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [26]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

# arquitetura da rede
arquitecture = [784, 30, 20, 10]
mlp = Network(arquitecture)
mlp.SGD(training_data, 10, 32, 0.5, test_data=test_data)

=== Configuração: [784, 32, 10] ===
Epoch 0 : 7088 / 10000 = 70.88%
Epoch 1 : 8254 / 10000 = 82.54%
Epoch 2 : 8567 / 10000 = 85.67%
Epoch 3 : 8703 / 10000 = 87.03%
Epoch 4 : 8767 / 10000 = 87.67%
Epoch 5 : 8787 / 10000 = 87.87%
Epoch 6 : 8818 / 10000 = 88.18%
Epoch 7 : 8833 / 10000 = 88.33%
Epoch 8 : 8837 / 10000 = 88.37%
Epoch 9 : 8839 / 10000 = 88.39%


##**Mini-Projeto**
1) Realizar avaliações modificando os seguintes parâmetros:     

     a) Taxa de aprendizagem: 0.1, 0.3 e 0.5
     b) Função de ativação RELU
     c) Rede com uma camada intermediária com 3 configurações diferentes (Explicite a configuração utilizada)
     d) Rede com duas camadas intermediárias com 3 configurações diferentes (Explicite a configuração utilizada)
     
      
Complete a Tabela abaixo com os resultados (Pra cada configuração de camadas intermediárias, execute as 3 taxas de aprendizagem pra função de ativação Sigmoide e Relu):


\begin{array}{|c|ccc|ccc|ccc|}\hline\\ \\
  1\;Camada\;Intermediária & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline
Configurações & 784 & 16 & 10 & 784 & 32 & 10 & 784 & 64 & 10 & \\ \hline
Sigmoide  & & 91.0\%;\hspace{5mm} 92.9\%;\hspace{5mm} 93.33\%;  & & & 91.78\%;\hspace{5mm} 93.57\%;\hspace{5mm} 94.41\%; & & & 92.15\%;\hspace{5mm} 94.08\%;\hspace{5mm} 94.83\%  &  & \\ \hline
Relu  & & 92.79\%;\hspace{5mm} 93.03\%;\hspace{5mm} 94.58\%; & & & 94.35\%;\hspace{5mm} 95.35\%;\hspace{5mm} 95.64\%; & & & 95.18\%;\hspace{5mm} 96.49\%;\hspace{5mm} 96.82\%; &  & \\ \hline
  2\;Camadas\;Intermediárias & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline
Configurações & 784 & 64\hspace{20mm}32 & 10 & 784 & 128\hspace{20mm}64 & 10 & 784 & 32\hspace{20mm}16 & 10 & \\ \hline
Sigmoide  & & 90.79\%;\hspace{5mm}94.31\%;\hspace{5mm}95.36\%; & &  & 91.1\%;\hspace{5mm}93.77\%;\hspace{5mm}95.45\%; & & & 90.7\%;\hspace{5mm}93.41\%;\hspace{5mm}94.65\%; &  & \\ \hline
Relu  & & 96.05\%;\hspace{5mm}96.1\%;\hspace{5mm}96.78\%; & &  & 96.73\%;\hspace{5mm}97.35\%;\hspace{5mm}97.69\%; & & & 94.6\%;\hspace{5mm}96.1\%;\hspace{5mm}95.67\%; &  & \\ \hline
\end{array}

2) Modifique a taxa de aprendizagem pra diminuir com o tempo. Execute com a melhor configuração encontrada. Melhorou o resultado?

Relu:
  - Em redes com 1 camada intermediária, há uma melhora no resultado da acurácia.
  - Já nas redes com 2 camadas intermediárias, a acurácia diminui um pouco quando a taxa de aprendizagem é 0.1 e 0.5, mas com a taxa 0.3, houve um aumento.

Sigmoide:
  - Em redes com 1 camada intermediária, há uma piora no resultado da acurácia.
  - Já nas redes com 2 camadas intermediárias, a acurácia diminui drasticamente, independente da taxa de aprendizagem.
